<h1 style="text-align: center;"> Homework 5 </h1>

<h3 style="text-align: center;">
[Textbook (adapted), Chapter 6, Problem 8]</br>
&&</br>
[Textbook (adapted), Chapter 6, Problem 10
</h3>


In [73]:
import gurobipy as gb

In [74]:
FEE = 600
PROFIT_MARGIN_PCT = 0.50

routes_data = [
    #C0   C1    C2  C3
    [276, 497, 251, 364], #Truck 0
    [179, 375, 298, 190], #Truck 1
    [150, 475, 344, 492], #Truck 2
    [99, 163, 285, 185],  #Truck 3 97 -> 99
    [305, 150, 225, 165]  #Truck 4

]

trucks = [i for i in range(5)] #row
cars = [i for i in range(4)] #col
capacity = [2, 1, 1, 1, 2] 

model1 = gb.Model()

routes = model1.addVars(trucks, cars, vtype=gb.GRB.BINARY)

model1.setObjective(
    gb.quicksum(
        routes_data[i][j]*routes[i, j]
        for i in trucks
        for j in cars
    )
)

for i in trucks:
    model1.addConstr(
        gb.quicksum(
            routes[i, j] for j in cars
        ) <= capacity[i]
    )

for j in cars:
    model1.addConstr(
        gb.quicksum(
            routes[i, j] for i in trucks
        ) == 1
    )

model1.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x01187466
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1490.0000000
Presolve time: 0.00s
Presolved: 9 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 918.0000000

Root relaxation: objective 6.640000e+02, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     664.0000000  664.00000  0.00%     -    0s

Explored 1 nodes (6 simplex it

In [75]:
for i in trucks:
    for j in cars:
        if routes[i, j].X:
            print(f"Truck {trucks[i]} picks up & delivers Car {cars[j]} (capacity: {capacity[i]})(data value: {routes_data[i][j]})")
print(f"Total Cost: ${model1.ObjVal}")
print(f"Total Profit: ${(FEE * len(cars) *PROFIT_MARGIN_PCT) - model1.ObjVal}")

Truck 1 picks up & delivers Car 3 (capacity: 1)(data value: 190)
Truck 3 picks up & delivers Car 0 (capacity: 1)(data value: 99)
Truck 4 picks up & delivers Car 1 (capacity: 2)(data value: 150)
Truck 4 picks up & delivers Car 2 (capacity: 2)(data value: 225)
Total Cost: $664.0
Total Profit: $536.0


In [76]:
## Model 2

model2 = gb.Model()

states = [
    "AZ",
    "CA",
    "CT",
    "GA",
    "IL",
    "MA",
    "ME",
    "MN",
    "MT",
    "NC",
    "NJ",
    "NV",
    "OH",
    "OR",
    "TX",
    "VA",
]

mail = [
    82380,
    212954,
    63796, 
    136562,
    296479,
    99070,
    38848,
    86207,
    33309,
    170997,
    104974,
    29608,
    260858,
    63605,
    214076,
    134692
]

mail_data = dict(zip(states, mail))

WEEKS = [i for i in range(6)]

week_1 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
week_2 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
week_3 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
week_4 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
week_5 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
week_6 = model2.addVars(len(mail_data), vtype = gb.GRB.BINARY)
max_mail = model2.addVar()

model2.setObjective(
    max_mail
)

model2.addConstr(
    gb.quicksum(
        week_1[i]*mail[i] for i in range(len(mail_data)) 
    ) <= max_mail
)

model2.addConstr(
    gb.quicksum(
        week_2[i]*mail[i] for i in range(len(mail_data)) 
    ) <= max_mail
)

model2.addConstr(
    gb.quicksum(
        week_3[i]*mail[i] for i in range(len(mail_data)) 
    ) <= max_mail
)

model2.addConstr(
    gb.quicksum(
        week_4[i]*mail[i] for i in range(len(mail_data)) 
    ) <= max_mail
)

model2.addConstr(
    gb.quicksum(
        week_5[i]*mail[i] for i in range(len(mail_data))
    ) <= max_mail
)

model2.addConstr(
    gb.quicksum(
        week_6[i]*mail[i] for i in range(len(mail_data)) 
    ) <= max_mail
)

for i in range(len(mail_data)):
    model2.addConstr(
        week_1[i] + week_2[i] + week_3[i] + week_4[i] + week_5[i] + week_6[i] >= 1
    )

model2.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 22 rows, 97 columns and 198 nonzeros
Model fingerprint: 0xdb52feb6
Variable types: 1 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 533832.00000
Presolve time: 0.00s
Presolved: 22 rows, 97 columns, 198 nonzeros
Variable types: 0 continuous, 97 integer (96 binary)

Root relaxation: objective 3.380692e+05, 32 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 338069.167    0    9 533832.000 338069.167  36.7%     -    0s
H    0     0                    403372.00000 338069.167  16.2%     -    0s

In [77]:
if model2.Status == gb.GRB.Status.OPTIMAL:
    for v in model2.getVars():
        if v.X == 1:
            print(f"{v.varname}.  State: {states[v.index % 16]}. Mail: {mail[v.index % 16]}")
print(f"Maximum amount of mail per week: {model2.objVal}")

C0.  State: AZ. Mail: 82380
C12.  State: OH. Mail: 260858
C20.  State: IL. Mail: 296479
C24.  State: MT. Mail: 33309
C34.  State: CT. Mail: 63796
C41.  State: NC. Mail: 170997
C42.  State: NJ. Mail: 104974
C51.  State: GA. Mail: 136562
C61.  State: OR. Mail: 63605
C63.  State: VA. Mail: 134692
C65.  State: CA. Mail: 212954
C69.  State: MA. Mail: 99070
C75.  State: NV. Mail: 29608
C86.  State: ME. Mail: 38848
C87.  State: MN. Mail: 86207
C94.  State: TX. Mail: 214076
Maximum amount of mail per week: 343238.0
